# Volumes-wise operation

This notebook demostrates common volumes-wise operations by using SDK.

## Prerequisites

You need

- an SDK environment set up with the administrator priviledge. Check "*environment setup.ipynb*" for the detail.

In [ ]:
PRIMEHUB_CLUSTER = 'your_PrimeHub_domain'

**Check if the configuration exists. If not, you need to login to retrieve the token and input it in the prompt. The configuration will be generated at**`~/.primehub/config.json`.

In [ ]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print(f"PrimeHub Python SDK {ph.version.version()} environment is ready, you are good to go.")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("Failed to retrieve the information from PrimeHub cluster, please check the configuration.")

## ToC

- [Fundamental Operations (CRUD)](#fundamental)
- [Common Operations](#common)

---
## Fundamental Operations <a class="anchor" id="fundamental"></a>

In [ ]:
# Get the group id by the given group name
group_name = 'Continental'
groups = ph.admin.groups.list()
for group in groups:
    if group['name'] == group_name:
        specified_group = group
        
group_detail = ph.admin.groups.get(specified_group['id'])
group_detail

### Create Volume

Method: `ph.admin.volumes.create(configuration)`

### Create Volume of type PV

A volume connecting to a persistent volumne

Yuo can opt for adding a *global volume (read-only)* and/or a *volume connecting specified groups with writable privildege*.

In [ ]:
persistent_volume_config_global = {
  "name": "pv-global-sdk",
  "displayName": "1GB global persistent volume",
  "description": "global persistent volume",
  "type": "pv",
  "global": True,
  "pvProvisioning": "auto",
  "volumeSize": 1,
  "groups": {
    "connect": [
      {
        "id": specified_group['id'],
        "writable": True
      }
    ]
  }
}

persistent_volume_config_with_specified_groups = {
  "name": "pv-groups-sdk",
  "displayName": "1GB persistent volume",
  "description": "persistent volume connecting groups",
  "type": "pv",
  "global": False,
  "pvProvisioning": "auto",
  "volumeSize": 1,
  "groups": {
    "connect": [
      {
        "id": specified_group['id'],
        "writable": True
      }
    ]
  }
}

In [ ]:
persistent_volume = ph.admin.volumes.create(persistent_volume_config_global)

ph.admin.volumes.get(persistent_volume["id"])

### Create Volume of type HostPath

A voloume connecting to a local path/directory. It is writable only to specified groups, read-only to others.

In [ ]:
# Create a host path volume with admin role
hostpath_config = {
  "name": "local-data-volume",
  "description": "created by sdk",
  "type": "hostPath",
  "global": True,
  "hostPath": "/opt/data",
  "groups": {
    "connect": [
      {
        "id": specified_group['id'],
        "writable": True
      }
    ]
  }
}

In [ ]:
hostpath_volume = ph.admin.volumes.create(hostpath_config)

ph.admin.volumes.get(hostpath_volume["id"]) 

### Create Volume of type Git Sync

A read-only volume connecting to a git repository

In [ ]:
# Create a git sync volume with admin role
gitsync_volume_config = {
  "name": "covid-19-dataset",
  "description": "created by sdk",
  "type": "git",
  "global": False,
  "url": "https://github.com/datasets/covid-19",
  #"secret": "secred_id" if required,
  "groups": {
    "connect": [
      {
        "id": specified_group['id'],
        "writable": False # must be False
      }
    ]
  }
}

In [ ]:
gitsync_volume = ph.admin.volumes.create(gitsync_volume_config)

ph.admin.volumes.get(gitsync_volume["id"]) 

### Create Volume of type Env
A read-only volume containing *environmental variables*

In [ ]:
ev_volume_config = {
  "name": "database-env",
  "description": "created by sdk",
  "type": "env",
  "global": False,
  "variables": {
    "PROD_MySQL": "The url to the production MySQL database",
    "STG_MySQL": "The url to the staging MySQL database"
  },
  "groups": {
    "connect": [
      {
        "id": specified_group['id'],
        "writable": False # must be False
      }
    ]
  }
}

In [ ]:
ev_volume = ph.admin.volumes.create(ev_volume_config)

ph.admin.volumes.get(ev_volume["id"])

### Get Volume Detail

Method: `ph.admin.volumes.get(volume_id)`


In [ ]:
ph.admin.volumes.get(persistent_volume['id'])

### Update Volume Configuration

Method: `ph.admin.volumes.update(configuration)`

In [ ]:
updating_config = {
  "variables": {
    "PROD_MySQL": "The url to the production MySQL database",
    "STG_MySQL": "The url to the staging MySQL database",
    "UAT_MySQL": "The url to the user acceptance testing MySQL database"
  }
}

In [ ]:
ph.admin.volumes.update(ev_volume['id'], updading_config)

ph.admin.volumes.get(ev_volume['id'])

### Get connected Groups of Volume

Method: `ph.admin.volumes.list_groups('volume_id')`

In [ ]:
ph.admin.volumes.list_groups(ev_volume['id'])

### List All of Volumes

Method: `ph.admin.volumes.list()`

In [ ]:
all_volumes = ph.admin.volumes.list()
for volume in all_volumes:
    print(volume)

### Delete Volume

Method: `ph.admin.volumes.delete(volume_id)`

In [ ]:
ph.admin.volumes.delete(ev_volume['id'])

---

## Common Operations <a class="anchor" id="common"></a>

### Connect/Disconnect Volume to Group

Method: 
- `ph.admin.volumes.add_group('volume_id', 'group_id')`
- `ph.admin.volumes.remove_group('volume_id', 'group_id')`

In [ ]:
volume_id = 'database-env'

ph.admin.volumes.add_group(volume_id, specified_group['id'])

#Verify
ph.admin.volumes.list_groups(volume_id)

In [ ]:
ph.admin.volumes.remove_group(volume_id, specified_group['id'])

#Verify
ph.admin.volumes.list_groups(volume_id)